In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Use /content/drive/MyDrive/ to save outputs to your Drive

Mounted at /content/drive


In [ ]:
# upgrade pip and install required packages
!pip install -U pip setuptools wheel
!pip install spacy
!pip install scispacy
# small scispaCy model (may take a minute)
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz
# fallback: small spaCy model (fast & lightweight)
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 105.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 141.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 117.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 140.2 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: blis
    Found existing installation: blis 1.3.0
    Uninstalling blis-1.3.0:
      Successfully uninstalled blis-1.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━ 5/8 [blis]^C


In [1]:
import spacy, pandas as pd
try:
    nlp = spacy.load("en_core_sci_sm")
    print("Loaded scispaCy en_core_sci_sm")
except Exception as e:
    print("scispaCy model not available — using spaCy en_core_web_sm instead:", e)
    nlp = spacy.load("en_core_web_sm")

scispaCy model not available — using spaCy en_core_web_sm instead: [E050] Can't find model 'en_core_sci_sm'. It doesn't seem to be a Python package or a valid path to a data directory.


In [2]:
reports = [
"""Patient Name: Riya Sharma
Age: 35
Diagnosis: Acute Bronchitis and Mild Asthma.
Prescribed Medications: Salbutamol Inhaler twice daily, Cetirizine 10mg once at night.
Test Reports: Chest X-ray - Mild inflammation observed.""",

"""Patient Name: Ajay Singh
Age: 58
Diagnosis: Type 2 Diabetes Mellitus and Hypertension.
Medications: Metformin 500 mg twice daily; Amlodipine 5 mg daily.
Recent Test: Fasting Blood Sugar = 162 mg/dL.""",

"""Patient Name: Sunita Patel
Age: 47
Complaint: Severe headache and photophobia.
Examination: CT scan normal.
Treatment: Started on Paracetamol 500mg as required; advised neurology follow-up."""
]


In [4]:
# Example if you have /content/drive/MyDrive/medical_reports.csv with column "report"
# df_in = pd.read_csv('/content/drive/MyDrive/medical_reports.csv')
# reports = df_in['report'].astype(str).tolist()

# Using the predefined reports list
print(f"Using {len(reports)} reports from the predefined list.")

Using 3 reports from the predefined list.


In [5]:
rows = []
for i, text in enumerate(reports, start=1):
    doc = nlp(text)
    # If model has custom labels, they appear in ent.label_
    if len(doc.ents)==0:
        # fallback simple token scanning (optional)
        pass
    for ent in doc.ents:
        rows.append({
            "report_id": i,
            "entity": ent.text,
            "label": ent.label_,
            "start_char": ent.start_char,
            "end_char": ent.end_char,
            "report_text": text
        })

results_df = pd.DataFrame(rows)
results_df

,report_id,entity,label,start_char,end_char,report_text
0,1,Riya Sharma,PERSON,14,25,Patient Name: Riya Sharma\nAge: 35\nDiagnosis:...
1,1,35,CARDINAL,31,33,Patient Name: Riya Sharma\nAge: 35\nDiagnosis:...
2,1,Mild Asthma,PERSON,66,77,Patient Name: Riya Sharma\nAge: 35\nDiagnosis:...
3,1,Salbutamol Inhaler,PERSON,103,121,Patient Name: Riya Sharma\nAge: 35\nDiagnosis:...
4,1,daily,DATE,128,133,Patient Name: Riya Sharma\nAge: 35\nDiagnosis:...
5,1,Cetirizine,PERSON,135,145,Patient Name: Riya Sharma\nAge: 35\nDiagnosis:...
6,1,10,CARDINAL,146,148,Patient Name: Riya Sharma\nAge: 35\nDiagnosis:...
7,1,night,TIME,159,164,Patient Name: Riya Sharma\nAge: 35\nDiagnosis:...
8,2,Ajay Singh\n,PERSON,14,25,Patient Name: Ajay Singh\nAge: 58\nDiagnosis: ...
9,2,58,CARDINAL,30,32,Patient Name: Ajay Singh\nAge: 58\nDiagnosis: ...


In [6]:
# group entities per report into a JSON-like structure
grouped = results_df.groupby('report_id').apply(
    lambda g: g[['entity','label']].to_dict(orient='records')
).to_dict()
grouped

/tmp/ipython-input-170546870.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = results_df.groupby('report_id').apply(


{1: [{'entity': 'Riya Sharma', 'label': 'PERSON'},
  {'entity': '35', 'label': 'CARDINAL'},
  {'entity': 'Mild Asthma', 'label': 'PERSON'},
  {'entity': 'Salbutamol Inhaler', 'label': 'PERSON'},
  {'entity': 'daily', 'label': 'DATE'},
  {'entity': 'Cetirizine', 'label': 'PERSON'},
  {'entity': '10', 'label': 'CARDINAL'},
  {'entity': 'night', 'label': 'TIME'}],
 2: [{'entity': 'Ajay Singh\n', 'label': 'PERSON'},
  {'entity': '58', 'label': 'CARDINAL'},
  {'entity': '2', 'label': 'CARDINAL'},
  {'entity': 'Hypertension', 'label': 'ORG'},
  {'entity': '500', 'label': 'CARDINAL'},
  {'entity': 'Amlodipine 5', 'label': 'PERSON'},
  {'entity': 'daily', 'label': 'DATE'},
  {'entity': '162', 'label': 'CARDINAL'}],
 3: [{'entity': 'Sunita Patel', 'label': 'PERSON'},
  {'entity': '47', 'label': 'CARDINAL'},
  {'entity': 'CT', 'label': 'ORG'}]}

In [8]:
import os

results_df.to_csv('/content/ner_results.csv', index=False)
results_df.to_json('/content/ner_results.json', orient='records')

# If Drive mounted, also save there for safe keeping:
drive_path = '/content/drive/MyDrive/NER'
if not os.path.exists(drive_path):
    os.makedirs(drive_path)
results_df.to_csv(os.path.join(drive_path, 'ner_results.csv'), index=False)